# Polymer Partitioning in Two Fluid System
---
This exercise is to demonstrate a simple workflow utilizing the entire set of MoSDeF tools to run a simulation in Cassandra. The main feature of this workflow is to demonstrate methods to use GMSO topologies instead of a ParmEd/OPENMM backend. This rerouted workflow, through a GMSO topology, gives the users the increased functionality and testing supported through GMSO. Although the workflow process remains relatively similar, small syntax differences allow users to gain increased functionality demonstrated in this workflow, such as using forcefield matching to individual molecules in the system. </br>

#### Note this workflow uses features that are still under development on the GMSO side of things. 
As such, it is necessary to install dev version of specific branches to access this functionality.
1. Install mBuild 
```python
git clone https://github.com/CalCraven/mbuild.git
git checkout top-down-molecule-id
pip install ./
```
2. Install GMSO
```python
git clone https://github.com/CalCraven/gmso.git
git checkout develop-gmsoff-parameterize
pip install ./
```
3. Install forcefield-utilities
```python
git clone https://github.com/CalCraven/forcefield-utilities.git
git checkout bugfix-ff-by-path
pip install ./
```
___


### Exercise Stages:
1. Import libraries
2. Custom mBuild recipes
3. Build partitioning box
4. Parameterize with multiple forcefields
5. Run Cassandra Simulations
6. Evaluate Results
---

# 1. Import Libraries
---

In [1]:
# Import Libraries
import numpy as np
import mbuild as mb
import forcefield_utilities as ffutils
import gmso

# 2. Custom mBuild Recipes
---

In [2]:
from scipy.constants import N_A
def Packing_Number(compound, vol):
    """Function to identify the number of compounds to place into a box."""
    n_compounds = compound.dens * vol / compound.mass * N_A * 1e-21
    return int(n_compounds)

In [3]:
import operator
import functools
def Partitioned_Box(solute, solvent1, solvent2, boxl, frac_interface=0):
    """
    Function to solubilize the solute into solvent 1 and fill other half of the 
    box with solvent2.
    """
    # Pack mbuild box with water, polymer, and hexane
    full_box = mb.Box([boxl, boxl, boxl])
    half_box = mb.Box([boxl/2, boxl, boxl])
    vol=functools.reduce(operator.mul, full_box.lengths, 1)/2
    solute.translate(np.array([boxl+boxl*frac_interface, boxl, boxl])/2)

    filled_box1 = mb.packing.solvate(
        solvent=solvent1, 
        solute=solute, 
        box=half_box,  
        n_solvent=Packing_Number(solvent1, vol),
        edge=0.01
    )
    filled_box2 = mb.packing.fill_box(
        compound=solvent2,
        box=half_box,  
        n_compounds=Packing_Number(solvent2, vol),
        edge=0.01
    )
    filled_box2.translate([boxl/2,0,0])
    partitioned_box = mb.Compound()
    partitioned_box.add(filled_box1)
    partitioned_box.add(filled_box2)
    return partitioned_box

# 3. Build Box of Molecules
---

In [4]:
polymer = mb.load("CCCCCCCCO", smiles=True, name="polymer")
polymer.name = "polymer"
cyclohexane = mb.load("C1CCCCC1", smiles=True, name="cyclohexane")
cyclohexane.name = "cyclohexane"
cyclohexane.dens = 0.63
cyclopentane = mb.load("C1CCCC1", smiles=True)
cyclopentane.name = "cyclopentane"
cyclopentane.dens = 0.63

boxl = 4
solute_position = 0
partitioned_box = Partitioned_Box(polymer, cyclopentane, cyclohexane, boxl, solute_position)
partitioned_box = partitioned_box.group_by_molecules()
partitioned_box.visualize()

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/mbuild/mbuild/compound.py:1249: UserWarning: Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box.
  warn(
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/mbuild/mbuild/compound.py:768: UserWarning: After adding new Compound, Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box
  warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# 4. Parameterize with Multiple Forcefields
---

## Load forcefields

In [5]:
import forcefield_utilities as ffutils
ffloader = ffutils.FoyerFFs()
alcohols_ff = ffloader.load("xmls/alcohols.xml").to_gmso_ff()
solvent_ff = ffloader.load("xmls/alkanes.xml").to_gmso_ff()
water_ff = ffloader.load("xmls/tip3p.xml").to_gmso_ff()

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/atom_type.py:180: UserWarning: Masses are assumed to be in units of g/mol
  warnings.warn(UNIT_WARNING_STRING.format("Masses", "g/mol"))
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/atom_type.py:191: UserWarning: Charges are assumed to be in units of elementary charge
  warnings.warn(


## Convert Topology to GMSO

In [6]:
from gmso.external import from_mbuild
topology_gmso = from_mbuild(partitioned_box)
topology_gmso.identify_connections() #Identify angles and dihedrals (this may be slow)

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/subtopology.py:70: UserWarning: Setting a parent is potentially dangerous. Consider using Topology.add_subtopology instead
  warnings.warn(


## Apply forcefields

In [7]:
from gmso.parameterization import apply
ff_dicts = {
    "cyclohexane": solvent_ff,
    "cyclopentane": solvent_ff,
    "polymer": alcohols_ff
} #The names here are from the molecule names that were put into the box, and can be found
#by looking at topology_gmso.subtops
apply(topology_gmso, ff_dicts)

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/forcefield.py:452: UserWarning: ImproperType between atoms CT, CT, HC and HC is missing from the ForceField.
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/forcefield.py:452: UserWarning: ImproperType between atoms opls_136, opls_136, opls_140 and opls_140 is missing from the ForceField.
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/forcefield.py:452: UserWarning: ImproperType between atoms opls_136, opls_135, opls_140 and opls_140 is missing from the ForceField.
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/forcefield.py:452: UserWarning: ImproperType between atoms CT, CT, CT and HC is missing from the ForceField.
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vande

<Topology Topology, 5214 sites,
 38211 connections,
 36477 potentials,
 id: 6080708816>

# 5. Run Cassandra Simulations
---

## Write output files

## Setup simulation configurations

## Run Simulations

# 6. Analyze Results
---